In [6]:
# %pip install --upgrade --quiet  llmlingua accelerate


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    # model_name="qwen2-1.5b-instruct",
    model_name="qwen2-plus",
    temperature=0,
    streaming=True,
)

ms_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('MOONSHOT_API_BASE'),
    openai_api_key=os.getenv('MOONSHOT_API_KEY'),
    model_name="moonshot-v1-8k",
    temperature=0,
    streaming=True,
)

cf_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('CF_API_BASE'),
    openai_api_key=os.getenv('CF_API_TOKEN'),
    model_name="@cf/meta/llama-3-8b-instruct",
    temperature=0,
    streaming=True,
)

groq_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('GROQ_API_BASE'),
    openai_api_key=os.getenv('GROQ_API_KEY'),
    model_name="llama3-8b-8192",
    temperature=0,
    streaming=True,
)

In [2]:
from langchain_community.embeddings import DashScopeEmbeddings
from dotenv import load_dotenv
import os

load_dotenv(override=True)
qw_embedding = DashScopeEmbeddings(
    model="text-embedding-v2", dashscope_api_key=os.getenv('DASHSCOPE_API_KEY')
)

In [3]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i + 1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [4]:
# 让我们开始初始化的一个简单的矢量储存的猎和储存2023年国家联盟的讲话(在块). 我们可以设立犬来检索数(20)的文档。

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

documents = TextLoader(
    "../../../../file/岭南味道 | 看醒狮打咏春，广东海陵岛玩水5日.md",
).load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

retriever = FAISS.from_documents(texts, qw_embedding).as_retriever(search_kwargs={"k": 20})

query = "What did the president say about Ketanji Brown Jackson"
docs = retriever.invoke(query)
pretty_print_docs(docs)

Document 1:

### D5(第5天)：

- 早餐后，退房前往顺德，首先游览岭南四大园林之一清晖园，园内处处体现主人移山水之中，尽赏曲径荷风之构想，游览时间约1小时。
  随后前往顺德著名的步行街华盖路，华盖路是一条有岭南特色的商业步行街，街道两旁为明末清初骑楼特色的岭南西洋建筑，彩色的骑楼、地道的美食店分布其中。
  午餐时间，大家自行华盖路的地道的美食店用餐。 下午适时集合上车，返回广州，先送广州南站，再送白云机场。
  坐高铁的游侠返程车次请订在16:30之后，坐飞机的游侠返程航班请订在18:30之后
- 车程：顺德-广州白云机场：76KM
- 用餐：含早，正餐自理
- 解散日：大约下午16：00返回广州，先送广州南站，再送白云机场，坐高铁的游侠返程车次请订在16:
  30之后，坐飞机的游侠返程航班请订在18:30之后。

## 费用说明
----------------------------------------------------------------------------------------------------
Document 2:

## 费用说明

- 费用包含：交通、住宿、餐食（全程4个酒店早餐）、门票及景交、体验项目、随团服务人员等。
- 儿童费用：交通标准同成人，部分门票及景交根据身高有优惠，不含住宿和早餐，现场超过身高现补门票。
- 自理费用：各地往返广州市的大交通费用、保险、全程正餐、景区里的二道门票或消费项目、单房差等。
- 赠送项目：游侠客定制头巾、精美徽章、每人每天一瓶水

## 活动说明

集合时间：每批第一天全天报到
集合地点：广州市区指定酒店
适合人群：适合4-65周岁，身体健康的旅行者
活动领队：游侠客广州领队（出发前3天出团通知）
成行人数：满6人成行，每批不超过25人

## 活动装备

- 证件：身份证必带，学生证、军官证等特殊证件
- 穿着：美美的休闲装，薄外套，帽，太阳镜，伞等
- 行李：行李箱+双肩包
- 药品：感冒药、肠胃药、维他命、风油精等自用药
- 数码：各类数码产品及充电器、摄影器材
- 护肤：防晒霜、保湿润肤水、唇膏等
- 洗漱：洗漱用品、换洗衣物

## 目的地介绍
---------------------------------------------------

In [5]:
# 现在我们总结我们的基猎犬带 ContextualCompressionRetriever 使用 LLMLinguaCompressor 作为一个压缩机。

from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import LLMLinguaCompressor


compressor = LLMLinguaCompressor(model_name="openai-community/gpt2", device_map="cpu")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)
pretty_print_docs(compressed_docs)

ImportError: Could not import llmlingua python package. Please install it with `pip install llmlingua`.

QA generation with LLMLingua

In [ ]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)

In [ ]:
chain.invoke({"query": query})